In [6]:
import yfinance as yf
import definition
from definition import give_df
import pandas as pd
import numpy as np


In [7]:
def get_er_dates(symbol):
        dfq = give_df(symbol)
        dfq["Binary_E"] = np.where(dfq['Event'] == "Earnings", 1, 0)
        #dfq["Binary_-1"] = np.where(dfq["Binary_E"].shift(-1) == 1, 1, 0)
        dfq["Pre-Market Change"] = dfq["Open"] - dfq["Close"].shift(+1)
        dfq["Intraday Change"] = dfq["Close"] - dfq["Open"]
        dfq["Pre-Market D+"] = np.where(dfq['Pre-Market Change']>0, 1, 0)
        dfq["Intraday D+"] = np.where(dfq['Intraday Change']>0, 1, 0)
        dfq["Pre-Market D-"] = np.where(dfq['Pre-Market Change']<0, 1, 0)
        dfq["Intraday D-"] = np.where(dfq['Intraday Change']<0, 1, 0)
        dfq["PreM and Intra +"] = np.where(dfq['Pre-Market D+'] +
                                           dfq["Intraday D+"] > 1, 1, 0)
        dfq["PreM+ and Intra -"] = np.where(dfq['Pre-Market D+'] +
                                           dfq["Intraday D-"] > 1, 1, 0)
        dfw = dfq.dropna()
        return dfw

In [8]:
stocks = ["TSLA", "AAPL"]

In [9]:
stockz = []
re1 = []
re2 = []
re3 = []

for i in stocks:
    print("Getting historical results for {}".format(i))
    dfw = pd.DataFrame(get_er_dates(i))
    name = i
    length_ = sum(dfw["Binary_E"])
    if length_ == 0:
        print("Skipping - No Data")
        pass
    else:
        result1 = round(((sum(dfw["PreM and Intra +"])/length_)*1),3)
        zero_denom_test = sum(dfw["Pre-Market D+"])
        if zero_denom_test == 0:
            result2 = 0
            continue
        else:
            result2 = round(((sum(dfw["PreM and Intra +"])/sum(dfw["Pre-Market D+"]))*1),3)
            result3 = round(((sum(dfw["PreM+ and Intra -"])/sum(dfw["Pre-Market D+"]))*1),3)
        stockz.append(name)
        re1.append(result1)
        re2.append(result2)
        re3.append(result3)

stock_parser = {
    "Symbol" : []    
}


print("Parsing results dataframe...")
results_df = pd.DataFrame(stock_parser)
results_df["Symbol"] = stockz
results_df["- increases in both premarket and intraday x% of the time."] = re1
results_df["- if increasing in premarket has a x% chance of increasing in intraday."] = re2
results_df["- if increasing in premarket has a x% change of falling in intraday."] = re3
label = "Premarket trading as % of daily action"
print("Saving...")
results_df.to_csv("Main Output.csv")
print("Done")

Getting historical results for TSLA
Getting historical results for AAPL
Parsing results dataframe...
Saving...
Done
